In [1]:
import tensorflow as tf
import keras
import numpy as np
import os
# to make this notebook's output stable across runs
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

# To plot pretty figures
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

import pandas as pd
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, LabelEncoder
import seaborn as sns

import scipy.stats as sc_stats

import random

from sklearn.model_selection import train_test_split
onehot_encoder=OneHotEncoder(sparse=False)

os.environ['KMP_DUPLICATE_LIB_OK']='True'


Using TensorFlow backend.


In [19]:
# parameters
time_steps = 1
inputs = 3
outputs = 2

In [20]:
def add_releveant_features(task_df):


    task_df['PrevOutcome']=task_df['Outcome'].shift(1)
    task_df.loc[1,'PrevOutcome']= 0

    task_df['PrevChoice']=task_df['Choice'].shift(1)
    task_df.loc[1,'PrevChoice']= 0

    task_df['PrevSafe']=task_df['Safe'].shift(1)
    task_df.loc[1,'PrevSafe']= 0

    task_df['PrevBigRisky']=task_df['BigRisky'].shift(1)
    task_df.loc[1,'PrevBigRisky']= 0

    task_df['PrevSmallRisky']=task_df['SmallRisky'].shift(1)
    task_df.loc[1,'PrevSmallRisky']= 0
    
    task_df['PrevRT']=task_df['RT'].shift(1)
    task_df.loc[1,'PrevRT']= 0
    
#     task_df['RPE'] =  task_df['Outcome'].shift(1) - 0.5*(task_df['BigRisky'].shift(1) + 
    
    return task_df

In [21]:
def add_kback_features(task_df):

    for k in range(1,11):
        task_df[str(k)+'backOutcome']=task_df['Outcome'].shift(k)
        task_df[str(k)+'backChoice']=task_df['Choice'].shift(k)
        task_df[str(k)+'backSafe']=task_df['Safe'].shift(k)
        task_df[str(k)+'backBigRisky']=task_df['BigRisky'].shift(k)
        task_df[str(k)+'backSmallRisky']=task_df['SmallRisky'].shift(k)

    return task_df

In [22]:
def chunk_split_data(data,start_chunk,end_chunk):
    
    a=[k for k in range(start_chunk,end_chunk)]
    out=[]

    for d in range(0,data.shape[0],20):

        c= [c+d for c in a]
        out = out+c

    while out[-1]>=data.shape[0]-1:
        out.pop()
#     return out
    return data[out]

In [23]:
def train_RNN(neurons,train_X,train_y,test_X,test_y,val_X,val_y): 
    reset_graph()

    learning_rate = 0.001
    epochs = 50000
    batch_size = int(train_X.shape[0]/2)
    # batch_size = 100
    length = train_X.shape[0]
    display = 100
    neurons = neurons

    num_batches = 100
    seq_len = 10

    percent_above_PT = 1

    train_threshold = 1.5#PT_R2 + percent_above_PT


    save_step = 100


    best_loss_val = np.infty
    checks_since_last_progress = 0
    max_checks_without_progress = 1000


    # clear graph (if any) before running
    tf.reset_default_graph()

    X = tf.placeholder(tf.float32, [None, time_steps, inputs])

    y = tf.placeholder(tf.float32, [None, outputs])

    # LSTM Cell
    cell = tf.contrib.rnn.BasicLSTMCell(num_units=neurons, activation=tf.nn.relu)
    cell_outputs, states = tf.nn.dynamic_rnn(cell, X, dtype=tf.float32)

    # pass into Dense layer
    stacked_outputs = tf.reshape(cell_outputs, [-1, neurons])
    out = tf.layers.dense(inputs=stacked_outputs, units=outputs)

    probability = tf.nn.softmax(out)

    # squared error loss or cost function for linear regression
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(
            labels=y, logits=out))

    # optimizer to minimize cost
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
    training_op = optimizer.minimize(loss)
    saver = tf.train.Saver()
    
    accuracy = tf.metrics.accuracy(labels =  tf.argmax(y, 1),
                          predictions = tf.argmax(out, 1),
                          name = "accuracy")
    precision = tf.metrics.precision(labels=tf.argmax(y, 1),
                                 predictions=tf.argmax(out, 1),
                                 name="precision")
    recall = tf.metrics.recall(labels=tf.argmax(y, 1),
                           predictions=tf.argmax(out, 1),
                           name="recall")
    f1 = 2 * accuracy[1] * recall[1] / ( precision[1] + recall[1] )

    acc_up,acc_val = accuracy
    auc = tf.metrics.auc(labels=tf.argmax(y, 1),
                           predictions=tf.argmax(out, 1),
                           name="auc")
    
    valid_store = []
    
    with tf.Session() as sess:
        #######################
#         saver.restore(sess, "./checkpts/Original_RNN_LSTM_8features_v2.ckpt")
#         saver.restore(sess, "./checkpts/OriginalDATA_RNN_LSTM_8features.ckpt")
        
        if pretraining == True:

            saver.restore(sess, "./checkpts/Original_v2_DATA_RNN_LSTM_8features.ckpt")

        #######################
        
        # initialize all variables
        tf.global_variables_initializer().run()
        tf.local_variables_initializer().run()

        # Train the model
        for steps in range(epochs):
            mini_batch = zip(range(0, length, batch_size),
                       range(batch_size, length+1, batch_size))

            # train data in mini-batches
            for (start, end) in mini_batch:
    #             print(start,end)
                sess.run(training_op, feed_dict = {X: train_X[start:end,:,:],
                                                   y: train_y[start:end,:]}) 

            ## train data in batches of length subsequence

    #         for k in range(num_batches):
    #             X_seq, y_seq = random_subsequence(train_X,train_y,seq_len)

    #             sess.run(training_op, feed_dict = {X:X_seq,y:y_seq}) 
            loss_fn = loss.eval(feed_dict = {X: train_X, y: train_y})
            loss_val = loss.eval(feed_dict={X: val_X, y: val_y})


            # print training performance 
            if (steps+1) % display == 0:
                # evaluate loss function on training set


                loss_fn = loss.eval(feed_dict = {X: train_X, y: train_y})
                print('Step: {}  \tTraining loss: {}'.format((steps+1), loss_fn))

                acc_train = acc_val.eval(feed_dict={X: train_X, y: train_y})
                print('Step: {}  \tTraining accuracy: {}'.format((steps+1), acc_train))


                acc_test = acc_val.eval(feed_dict={X: test_X, y: test_y})
    #             print('Step: {}  \tTest accuracy: {}'.format((steps+1), acc_test))

                loss_test = loss.eval(feed_dict={X: test_X, y: test_y})
    #             print('Step: {}  \tTest loss: {}'.format((steps+1), loss_test))

                accu_val = acc_val.eval(feed_dict={X: val_X, y: val_y})

                loss_val = loss.eval(feed_dict={X: val_X, y: val_y})
                print('Step: {}  \tValid loss: {}'.format((steps+1), loss_val))

                valid_store.append(loss_val)







            if (1 + loss_fn/np.log(0.5)) > train_threshold:
                    print("Threshold achieved, quit training")
                    break


            if loss_val < best_loss_val:

                        best_loss_val = loss_val
                        checks_since_last_progress = 0
            else:
                            checks_since_last_progress += 1


            # EARLY STOPPING
            if checks_since_last_progress > max_checks_without_progress:
                print("Early stopping!")
                break


            if (steps+1) % save_step ==0:
                                save_path = saver.save(sess, "./checkpts/Later_RNN_LSTM_8features.ckpt")

#                 save_path = saver.save(sess, "./checkpts/RNN_Internet_LSTM_model_5features.ckpt")





    #     evaluate model accuracy
        acc, prec, recall, f1, AUC = sess.run([accuracy, precision, recall, f1,auc],
                                         feed_dict = {X: train_X, y: train_y})
        prob_train = probability.eval(feed_dict = {X: train_X, y: train_y})
        prob_test = probability.eval(feed_dict = {X: test_X, y: test_y})
        prob_valid = probability.eval(feed_dict = {X: val_X, y: val_y})



        print('\nEvaluation  on training set')
        print('Accuracy:', acc[1])
        print('Precision:', prec[1])
        print('Recall:', recall[1])
        print('F1 score:', f1)
        print('AUC:', AUC[1])
        
      
    
    
#         save_path = saver.save(sess, "./checkpts/Original_v2_DATA_RNN_LSTM_8features.ckpt")
#         save_path = saver.save(sess, "./checkpts/Later_v2_DATA_RNN_LSTM_8features.ckpt")

        
#         save_path = saver.save(sess, "./checkpts/OriginalDATA_RNN_LSTM_8features.ckpt")
#         save_path = saver.save(sess, "./checkpts/LaterDATA_RNN_LSTM_8features.ckpt")


#         save_path = saver.save(sess, "./checkpts/Original_RNN_LSTM_8features.ckpt")

#         save_path = saver.save(sess, "./checkpts/Later_RNN_LSTM_8features.ckpt")




    ## APP DATA
#         save_path = saver.save(sess, "./checkpts/Original_v2_APPDATA_RNN_LSTM_8features.ckpt")
        save_path = saver.save(sess, "./checkpts/Later_v2_APPDATA_RNN_LSTM_8features.ckpt")


    metric_out_df= pd.DataFrame(np.array([acc[1],prec[1],recall[1],f1,AUC[1],loss_fn,accu_val,best_loss_val,acc_test,loss_test,neurons,learning_rate,epochs,steps]).reshape(-1,14),columns =["accuracy","precision","recall","f1_score","auc","loss","accuracy_val","loss_val","accuracy_test","loss_test","neurons","learning_rate","n_epochs","steps"])
    return metric_out_df, prob_train, prob_test, prob_valid
    


    
    
    
    
def random_subsequence(X,y,seq_len):
    rnd  = random.randint(0,len(X)-seq_len)
    X_seq, y_seq = X[rnd:rnd+seq_len,:], y[rnd:rnd+seq_len,:]
    return X_seq, y_seq

    print(y_seq.shape)

## Odd plays train, even plays test and valid


### Generate subject files

In [24]:
dir_list = os.listdir("/Users/ritwik7/Dropbox (Personal)/Postdoc_UCL/DATA/rlab_incomplete_rewardSWB_code/by_RN/appdata/")
dir_path ="/Users/ritwik7/Dropbox (Personal)/Postdoc_UCL/DATA/rlab_incomplete_rewardSWB_code/by_RN/appdata/"

subj_files_list =[]; ## list of subject_files fullfilling a criteria

dir_files = [i for i in os.listdir(dir_path) if i.startswith('sub')]

for subj_file_path in dir_files:

    file_path  ="/Users/ritwik7/Dropbox (Personal)/Postdoc_UCL/DATA/rlab_incomplete_rewardSWB_code/by_RN/appdata/"+ subj_file_path
    mypath =file_path
    
    play_names = [i for i in os.listdir(mypath) if os.path.isfile(os.path.join(mypath,i)) and i.startswith('app')]   
    
    if len(play_names) >= 50: ## criteria
        subj_files_list.append(subj_file_path)
        
    

In [25]:
len(subj_files_list)

37

### generate train, valid and test data

In [9]:
# num_shuffles=1
# for num, subj_file_path in enumerate(subj_files_list):
#     print(num)
# # for subj_file_path in [subj_files_list[0]]:
    
# #     train_data,test_data, val_data = np.empty((0,task_df.columns.shape[0])),  np.empty((0,task_df.columns.shape[0])), np.empty((0,task_df.columns.shape[0]))
#     train_data,test_data, val_data = np.empty((0,15)),  np.empty((0,15)), np.empty((0,15))

#     file_path  ="/Users/ritwik7/Dropbox (Personal)/Postdoc_UCL/DATA/rlab_incomplete_rewardSWB_code/by_RN/appdata/"+ subj_file_path
#     mypath =file_path
        
#     comp_task_train_df = pd.DataFrame()

#     play_names = [i for i in os.listdir(mypath) if os.path.isfile(os.path.join(mypath,i)) and i.startswith('app')]    

#     for randomization_counter in range(0,num_shuffles):
#             randomized_play_names= random.sample(play_names,len(play_names))
            
#             for play_num, play_name in enumerate(randomized_play_names):
# #         for play_num,play_name in enumerate(play_names):

#                 file_name = file_path + "/" + play_name
#                 task_df = pd.read_csv(file_name)
#                 task_df = add_releveant_features(task_df)

#                 if np.mod(play_num,2)==0: ## odd trials
#                     train_data = np.append(train_data,task_df[task_df.TrialNum>1].values, axis=0)

#                 else:
#                     test_data =  np.append(test_data, task_df[task_df.TrialNum>1].values[0:16], axis=0)
#                     val_data =  np.append(val_data, task_df[task_df.TrialNum>1].values[16:], axis=0)


#     train_data_df= pd.DataFrame(train_data,columns=task_df.columns)
#     val_data_df = pd.DataFrame(test_data,columns=task_df.columns)
#     test_data_df= pd.DataFrame(val_data,columns=task_df.columns)

# #     file_path = file_path + "/OddEvenPlays/"
#     file_path = file_path + "/OddEvenPlays/RandomizedPlays1"

# #     os.mkdir(file_path)
#     train_data_df.to_csv(file_path+"/train_data.csv")
#     test_data_df.to_csv(file_path+"/test_data.csv")
#     val_data_df.to_csv(file_path+"/val_data.csv")

### When Randomizing Trials

In [10]:
def randomize_trials(df):

    locs= random.sample([a for a in range(0,df.shape[0])],df.shape[0])
    # len(locs)
    df = add_releveant_features(df.loc[locs])
    
    ### get rid of first index since it contains NaN for previous trials
    df  = df.iloc[1:]
    
    return df , pd.DataFrame(locs)

In [11]:
def generate_randomizations(train_data_df,test_data_df,val_data_df):
        train_data_df,train_locs = randomize_trials(train_data_df)
        test_data_df,test_locs = randomize_trials(test_data_df)
        val_data_df,val_locs = randomize_trials(val_data_df)
        
        #         os.mkdir(file_path)
        train_locs.to_csv(file_path+"/train_locs.csv")
        test_locs.to_csv(file_path+"/test_locs.csv")
        val_locs.to_csv(file_path+"/val_locs.csv")
        
        return train_locs, test_locs, val_locs

In [12]:
def get_shuffled_data():
        train_locs = pd.read_csv(file_path+"/train_locs.csv")
        test_locs = pd.read_csv(file_path+"/test_locs.csv")
        val_locs = pd.read_csv(file_path+"/val_locs.csv")
        train_data_random_df  = train_data_df.iloc[train_locs.iloc[:,1]]
        test_data_random_df  = test_data_df.iloc[test_locs.iloc[:,1]]
        val_data_random_df  = val_data_df.iloc[val_locs.iloc[:,1]]
        
        train_data_random_df = add_releveant_features(train_data_random_df)
        train_data_random_df = add_kback_features(train_data_random_df)
        test_data_random_df = add_releveant_features(test_data_random_df)
        test_data_random_df = add_kback_features(test_data_random_df)
        val_data_random_df = add_releveant_features(val_data_random_df)
        val_data_random_df = add_kback_features(val_data_random_df)
        
        return train_data_random_df, test_data_random_df,val_data_random_df, train_locs, test_locs, val_locs 

In [26]:
def restore_input_feature(randomize_trials_flag,hist_flag):

    if randomize_trials_flag==True:
        if hist_flag==0: ## CURR OPTIONS ONLY
            print("return as is")
            
    
        if hist_flag==1: ## CURR OPTIONS, PREV ACTIONS:
            train_data_random_df.loc[train_locs.iloc[:,1],'PrevChoice'] = train_data_df.loc[train_locs.iloc[:,1],'PrevChoice']
            test_data_random_df.loc[test_locs.iloc[:,1],'PrevChoice'] = test_data_df.loc[test_locs.iloc[:,1],'PrevChoice']
            val_data_random_df.loc[val_locs.iloc[:,1],'PrevChoice'] = val_data_df.loc[val_locs.iloc[:,1],'PrevChoice']



        elif hist_flag==2: # CURR OPTIONS, PREV OUTCOME        

            train_data_random_df.loc[train_locs.iloc[:,1],'PrevOutcome'] = train_data_df.loc[train_locs.iloc[:,1],'PrevOutcome']
            test_data_random_df.loc[test_locs.iloc[:,1],'PrevOutcome'] = test_data_df.loc[test_locs.iloc[:,1],'PrevOutcome']
            val_data_random_df.loc[val_locs.iloc[:,1],'PrevOutcome'] = val_data_df.loc[val_locs.iloc[:,1],'PrevOutcome']


        elif hist_flag==3: ## CURR OPTIONS, PREV ACTIONS, PREV OUTCOME

            train_data_random_df.loc[train_locs.iloc[:,1],'PrevChoice','PrevOutcome'] = train_data_df.loc[train_locs.iloc[:,1],'PrevChoice','PrevOutcome']
            test_data_random_df.loc[test_locs.iloc[:,1],'PrevChoice','PrevOutcome'] = test_data_df.loc[test_locs.iloc[:,1],'PrevChoice','PrevOutcome']
            val_data_random_df.loc[val_locs.iloc[:,1],'PrevChoice','PrevOutcome'] = val_data_df.loc[val_locs.iloc[:,1],'PrevChoice','PrevOutcome']



    ####### Prev O + C+ R + CurrO--------------------
        elif hist_flag==4: # CURR OPTIONS, PREV ACTIONS, PREV OUTCOME, PREV OPTIONS
            print(hist_flag)
   
    return train_data_random_df, test_data_random_df,val_data_random_df 

### Split and format data

In [27]:
def data_split_odd_even(train_data_df,test_data_df,val_data_df):

#     train_len = 29
#     test_len = 14
#     val_len = 15

    ##----------------- UNCOMMENT BELOW
    
    if hist_flag==0: ## CURR OPTIONS ONLY
        
    
        train_X = train_data_df[['Safe','BigRisky','SmallRisky']].values
        train_y = train_data_df[['Choice']].values.astype(np.int32)

        test_X = test_data_df[['Safe','BigRisky','SmallRisky']].values
        test_y = test_data_df[['Choice']].values.astype(np.int32)

        val_X = val_data_df[['Safe','BigRisky','SmallRisky']].values
        val_y = val_data_df[['Choice']].values.astype(np.int32)

    elif hist_flag==1: ## CURR OPTIONS, PREV ACTIONS:
        
        train_X = train_data_df[['Safe','BigRisky','SmallRisky','PrevChoice']].values
        train_y = train_data_df[['Choice']].values.astype(np.int32)

        test_X = test_data_df[['Safe','BigRisky','SmallRisky','PrevChoice']].values
        test_y = test_data_df[['Choice']].values.astype(np.int32)

        val_X = val_data_df[['Safe','BigRisky','SmallRisky','PrevChoice']].values
        val_y = val_data_df[['Choice']].values.astype(np.int32)
        
        
      
    elif hist_flag==2: # CURR OPTIONS, PREV OUTCOME        
        train_X = train_data_df[['Safe','BigRisky','SmallRisky','PrevOutcome']].values
        train_y = train_data_df[['Choice']].values.astype(np.int32)

        test_X = test_data_df[['Safe','BigRisky','SmallRisky','PrevOutcome']].values
        test_y = test_data_df[['Choice']].values.astype(np.int32)

        val_X = val_data_df[['Safe','BigRisky','SmallRisky','PrevOutcome']].values
        val_y = val_data_df[['Choice']].values.astype(np.int32)
       
        
        
        
        
        
        
        
    elif hist_flag==3: ## CURR OPTIONS, PREV ACTIONS, PREV OUTCOME
        
        train_X = train_data_df[['Safe','BigRisky','SmallRisky','PrevChoice','PrevOutcome']].values
        train_y = train_data_df[['Choice']].values.astype(np.int32)

        test_X = test_data_df[['Safe','BigRisky','SmallRisky','PrevChoice','PrevOutcome']].values
        test_y = test_data_df[['Choice']].values.astype(np.int32)

        val_X = val_data_df[['Safe','BigRisky','SmallRisky','PrevChoice','PrevOutcome']].values
        val_y = val_data_df[['Choice']].values.astype(np.int32)
             
        

####### Prev O + C+ R + CurrO--------------------
    elif hist_flag==4: # CURR OPTIONS, PREV ACTIONS, PREV OUTCOME, PREV OPTIONS
        
        train_X = train_data_df[['Safe','BigRisky','SmallRisky','PrevOutcome','PrevChoice','PrevSafe','PrevBigRisky','PrevSmallRisky']].values
        train_y = train_data_df[['Choice']].values.astype(np.int32)

        test_X = test_data_df[['Safe','BigRisky','SmallRisky','PrevOutcome','PrevChoice','PrevSafe','PrevBigRisky','PrevSmallRisky']].values
        test_y = test_data_df[['Choice']].values.astype(np.int32)

        val_X = val_data_df[['Safe','BigRisky','SmallRisky','PrevOutcome','PrevChoice','PrevSafe','PrevBigRisky','PrevSmallRisky']].values
        val_y = val_data_df[['Choice']].values.astype(np.int32)
    
    
    
    
    

    
    ######## sampling 
    
    
#### - Prev RT+C+R+O + Curr O----------------------

#     train_X = task_df.loc[task_df.TrialNum>1, ['Safe','BigRisky','SmallRisky','PrevOutcome','PrevChoice','PrevSafe','PrevBigRisky','PrevSmallRisky','PrevRT']].values
#     train_y = task_df.loc[task_df.TrialNum>1,['Choice']].values.astype(np.int32)

#     test_X = dopa_task_df.loc[dopa_task_df.TrialNum>1,['Safe','BigRisky','SmallRisky','PrevOutcome','PrevChoice','PrevSafe','PrevBigRisky','PrevSmallRisky','PrevRT']].values
#     test_y = dopa_task_df.loc[dopa_task_df.TrialNum>1,['Choice']].values.astype(np.int32)






    #### PRE TRAINING
#     stop = int(0.7*len(train_X))
#     print(stop)
#     train_X, test_X, val_X, train_y, test_y, val_y= train_X[:stop], train_X[stop:stop+int((len(train_X)-stop)/2)], train_X[stop+int((len(train_X)-stop)/2):],train_y[:stop], train_y[stop:stop+int((len(train_X)-stop)/2)], train_y[stop+int((len(train_X)-stop)/2):]
    
#     train_X, test_X, val_X, train_y, test_y, val_y = train_X, test_X, test_X, train_y, test_y, test_y
    ###################################################################


    print(train_X.shape)
    print(train_y.shape)
    print(val_X.shape)
    print(val_y.shape)
    print(test_X.shape)
    print(test_y.shape)

    # # center and scale
    scaler = MinMaxScaler(feature_range=(0, 1))    
    train_X = scaler.fit_transform(train_X)
    test_X = scaler.fit_transform(test_X)
    val_X = scaler.fit_transform(val_X)


    train_X = train_X[:,None,:]
    val_X = val_X[:,None,:]
    test_X = test_X[:,None,:]


    # # one-hot encode the outputs

    onehot_encoder = OneHotEncoder()
    encode_categorical = train_y.reshape(len(train_y), 1)
    encode_categorical_test = test_y.reshape(len(test_y), 1)
    encode_categorical_val = val_y.reshape(len(val_y),1)


    train_y = onehot_encoder.fit_transform(encode_categorical).toarray()
    test_y = onehot_encoder.fit_transform(encode_categorical_test).toarray()
    val_y = onehot_encoder.fit_transform(encode_categorical_val).toarray()

    
    return train_X, train_y, test_X, test_y, val_X,val_y


### Run main training and save output

In [28]:
neurons = 8
hist_flag=0
randomize_trials_flag=True


for num, subj_file_path in enumerate(subj_files_list):
    print(num)

# for subj_file_path in [subj_files_list[0]]:

    file_path  ="/Users/ritwik7/Dropbox (Personal)/Postdoc_UCL/DATA/rlab_incomplete_rewardSWB_code/by_RN/appdata/"+ subj_file_path
                
#     file_path = file_path + "/OddEvenPlays"
    file_path = file_path + "/OddEvenPlays/RandomizedPlays1"



    train_data_df= pd.read_csv(file_path+"/train_data.csv")
    test_data_df = pd.read_csv(file_path+"/test_data.csv")
    val_data_df = pd.read_csv(file_path+"/val_data.csv")
    
    
    ####### IMPORTANT ALTERATION
    train_data_df = add_kback_features(train_data_df)
    val_data_df = add_kback_features(val_data_df)
    test_data_df = pd.read_csv(file_path+"/test_data_new.csv") ## otherwise testdata df would have NaNs for 1 back

    if randomize_trials_flag==True:
        file_path = file_path + "/RandomizeTrials"

        #### FOR GENERATING RANDOMIZATIONS
#         train_locs, test_locs, val_locs = generate_randomizations(train_data_df,test_data_df,val_data_df)

        train_data_random_df, test_data_random_df,val_data_random_df, train_locs, test_locs, val_locs  = get_shuffled_data()

        train_data_random_df, test_data_random_df,val_data_random_df = restore_input_feature(randomize_trials_flag,hist_flag)
        
        train_data_df , test_data_df, val_data_df = train_data_random_df, test_data_random_df, val_data_random_df
        
        
        
        
    
    print(file_path)
    
    
    train_X, train_y, test_X, test_y,val_X,val_y = data_split_odd_even(train_data_df,test_data_df,val_data_df)

    pretraining = False; 
    metric_out_df, prob_train, prob_test, prob_val = train_RNN(neurons,train_X,train_y,test_X,test_y,val_X,val_y)
    
    print(metric_out_df)
    
   
 
    prob_train_df = pd.DataFrame(prob_train.reshape(-1,2),columns = {'action_0','action_1'})
    prob_test_df = pd.DataFrame(prob_test.reshape(-1,2),columns = {'action_0','action_1'})
    prob_val_df = pd.DataFrame(prob_val.reshape(-1,2),columns = {'action_0','action_1'})

    if hist_flag==0:
        metric_out_df.to_csv(file_path+"/LSTM_updated_Crossval_currO_metricsneurons="+str(neurons)+".csv")
        prob_train_df.to_csv(file_path + "/prob_train_currO_neurons="+str(neurons)+".csv")
        prob_test_df.to_csv(file_path + "/prob_test_currO_neurons="+str(neurons)+".csv")
        prob_val_df.to_csv(file_path + "/prob_val_currO_neurons="+str(neurons)+".csv")
    
    
    elif hist_flag==1:
        metric_out_df.to_csv(file_path+"/LSTM_updated_Crossval_currOprevC_metricsneurons="+str(neurons)+".csv")
        prob_train_df.to_csv(file_path + "/prob_train_currOprevC_neurons="+str(neurons)+".csv")
        prob_test_df.to_csv(file_path + "/prob_test_currOprevC_neurons="+str(neurons)+".csv")
        prob_val_df.to_csv(file_path + "/prob_val_currOprevC_neurons="+str(neurons)+".csv")
        
        
    elif hist_flag==2:
        metric_out_df.to_csv(file_path+"/LSTM_updated_Crossval_currOprevR_metricsneurons="+str(neurons)+".csv")
        prob_train_df.to_csv(file_path + "/prob_train_currOprevR_neurons="+str(neurons)+".csv")
        prob_test_df.to_csv(file_path + "/prob_test_currOprevR_neurons="+str(neurons)+".csv")
        prob_val_df.to_csv(file_path + "/prob_val_currOprevR_neurons="+str(neurons)+".csv")

    elif hist_flag==3:
        metric_out_df.to_csv(file_path+"/LSTM_updated_Crossval_currOprevRC_metricsneurons="+str(neurons)+".csv")
        prob_train_df.to_csv(file_path + "/prob_train_currOprevRC_neurons="+str(neurons)+".csv")
        prob_test_df.to_csv(file_path + "/prob_test_currOprevRC_neurons="+str(neurons)+".csv")
        prob_val_df.to_csv(file_path + "/prob_val_currOprevRC_neurons="+str(neurons)+".csv")
# ################################
    elif hist_flag==4:
        metric_out_df.to_csv(file_path+"/LSTM_updated_Crossval_currprev_opts_metricsneurons="+str(neurons)+".csv")
        prob_train_df.to_csv(file_path + "/prob_train_currentprevopts_neurons="+str(neurons)+".csv")
        prob_test_df.to_csv(file_path + "/prob_test_currentprevopts_neurons="+str(neurons)+".csv")
        prob_val_df.to_csv(file_path + "/prob_val_currentprevopts_neurons="+str(neurons)+".csv")
# #############################

0
return as is
/Users/ritwik7/Dropbox (Personal)/Postdoc_UCL/DATA/rlab_incomplete_rewardSWB_code/by_RN/appdata/subject_num_27883/OddEvenPlays/RandomizedPlays1/RandomizeTrials
(1189, 3)
(1189, 1)
(640, 3)
(640, 1)
(520, 3)
(520, 1)


/Users/ritwik7/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/Users/ritwik7/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='au

Step: 100  	Training loss: 0.4570558965206146
Step: 100  	Training accuracy: 0.8696383237838745
Step: 100  	Valid loss: 0.4564363956451416
Step: 200  	Training loss: 0.4131602346897125
Step: 200  	Training accuracy: 0.8651780486106873
Step: 200  	Valid loss: 0.41126784682273865
Step: 300  	Training loss: 0.40119293332099915
Step: 300  	Training accuracy: 0.8642772436141968
Step: 300  	Valid loss: 0.40256696939468384
Step: 400  	Training loss: 0.3928319811820984
Step: 400  	Training accuracy: 0.8638902306556702
Step: 400  	Valid loss: 0.3980369567871094
Step: 500  	Training loss: 0.3882521688938141
Step: 500  	Training accuracy: 0.8636749982833862
Step: 500  	Valid loss: 0.3970620036125183
Step: 600  	Training loss: 0.3863036334514618
Step: 600  	Training accuracy: 0.8635379672050476
Step: 600  	Valid loss: 0.39786943793296814
Step: 700  	Training loss: 0.3855426013469696
Step: 700  	Training accuracy: 0.8634430170059204
Step: 700  	Valid loss: 0.3988562822341919
Step: 800  	Training lo

/Users/ritwik7/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/Users/ritwik7/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='au

Step: 100  	Training loss: 0.5664170980453491
Step: 100  	Training accuracy: 0.7593244314193726
Step: 100  	Valid loss: 0.5785761475563049
Step: 200  	Training loss: 0.5290668606758118
Step: 200  	Training accuracy: 0.7523652911186218
Step: 200  	Valid loss: 0.5527979135513306
Step: 300  	Training loss: 0.5248906016349792
Step: 300  	Training accuracy: 0.7509734630584717
Step: 300  	Valid loss: 0.5487770438194275
Step: 400  	Training loss: 0.5169987678527832
Step: 400  	Training accuracy: 0.7503769993782043
Step: 400  	Valid loss: 0.5403470396995544
Step: 500  	Training loss: 0.5024215579032898
Step: 500  	Training accuracy: 0.7500455975532532
Step: 500  	Valid loss: 0.5243799090385437
Step: 600  	Training loss: 0.48167943954467773
Step: 600  	Training accuracy: 0.7498347163200378
Step: 600  	Valid loss: 0.5014874935150146
Step: 700  	Training loss: 0.4592260420322418
Step: 700  	Training accuracy: 0.7496887445449829
Step: 700  	Valid loss: 0.47566452622413635
Step: 800  	Training loss

...

Step: 100  	Training loss: 0.42301568388938904
Step: 100  	Training accuracy: 0.8316429853439331
Step: 100  	Valid loss: 0.47548243403434753
Step: 200  	Training loss: 0.4086352288722992
Step: 200  	Training accuracy: 0.8272481560707092
Step: 200  	Valid loss: 0.4681565463542938
Step: 300  	Training loss: 0.39529120922088623
Step: 300  	Training accuracy: 0.8271805047988892
Step: 300  	Valid loss: 0.45572352409362793
Step: 400  	Training loss: 0.3827185332775116
Step: 400  	Training accuracy: 0.8287453055381775
Step: 400  	Valid loss: 0.4446384608745575
Step: 500  	Training loss: 0.3721393942832947
Step: 500  	Training accuracy: 0.8300653696060181
Step: 500  	Valid loss: 0.4354742169380188
Step: 600  	Training loss: 0.3636947572231293
Step: 600  	Training accuracy: 0.8321040272712708
Step: 600  	Valid loss: 0.4279986023902893
Step: 700  	Training loss: 0.35705360770225525
Step: 700  	Training accuracy: 0.8345295786857605
Step: 700  	Valid loss: 0.4219512939453125
Step: 800  	Training l

/Users/ritwik7/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/Users/ritwik7/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='au

Step: 100  	Training loss: 0.556518018245697
Step: 100  	Training accuracy: 0.870751678943634
Step: 100  	Valid loss: 0.5574180483818054
Step: 200  	Training loss: 0.37118545174598694
Step: 200  	Training accuracy: 0.8668651580810547
Step: 200  	Valid loss: 0.3757796585559845
Step: 300  	Training loss: 0.3101937770843506
Step: 300  	Training accuracy: 0.866087794303894
Step: 300  	Valid loss: 0.3183519244194031
Step: 400  	Training loss: 0.2925388813018799
Step: 400  	Training accuracy: 0.8657546639442444
Step: 400  	Valid loss: 0.3020056486129761
Step: 500  	Training loss: 0.28303462266921997
Step: 500  	Training accuracy: 0.8655695915222168
Step: 500  	Valid loss: 0.29263052344322205
Step: 600  	Training loss: 0.27518320083618164
Step: 600  	Training accuracy: 0.8660833239555359
Step: 600  	Valid loss: 0.28458210825920105
Step: 700  	Training loss: 0.26784178614616394
Step: 700  	Training accuracy: 0.8671640753746033
Step: 700  	Valid loss: 0.2769226133823395
Step: 800  	Training los

/Users/ritwik7/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/Users/ritwik7/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='au

Step: 100  	Training loss: 0.4825937747955322
Step: 100  	Training accuracy: 0.8332160711288452
Step: 100  	Valid loss: 0.4921784996986389
Step: 200  	Training loss: 0.4605080187320709
Step: 200  	Training accuracy: 0.8400187492370605
Step: 200  	Valid loss: 0.4726478159427643
Step: 300  	Training loss: 0.4488756060600281
Step: 300  	Training accuracy: 0.8413792848587036
Step: 300  	Valid loss: 0.4586648643016815
Step: 400  	Training loss: 0.43960514664649963
Step: 400  	Training accuracy: 0.8419623970985413
Step: 400  	Valid loss: 0.447162389755249
Step: 500  	Training loss: 0.4309469163417816
Step: 500  	Training accuracy: 0.8422863483428955
Step: 500  	Valid loss: 0.4365220367908478
Step: 600  	Training loss: 0.41991204023361206
Step: 600  	Training accuracy: 0.8424924612045288
Step: 600  	Valid loss: 0.423277884721756
Step: 700  	Training loss: 0.4065781533718109
Step: 700  	Training accuracy: 0.8426352143287659
Step: 700  	Valid loss: 0.4081227481365204
Step: 800  	Training loss: 

In [138]:
# df_out = randomize_trials(train_data_df)

In [143]:
# train_locs.iloc[0:5,1]

In [144]:
# train_data_random_df[["PrevChoice","PrevOutcome"]]

In [141]:
# df_out.head(10)

In [145]:
# test_data_random_df[["PrevChoice","PrevOutcome"]]